In [1]:
# Importando as bibliotecas necessárias:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import matplotlib.pyplot as plt
import logging
import os

In [2]:
# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
# Função para calcular o MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [5]:
# Carregar os dados
logging.info("Carregando dados...")
data_path = os.path.join("..", "data", "Dados_TCC_2014_2024_Normalizados_TabUnica.xlsx")
data = pd.read_excel(data_path)

2025-07-22 00:32:50,052 - INFO - Carregando dados...


In [6]:
# Preparar os dados
logging.info("Preparando dados...")
data['IPCA_Next'] = data['Valor IPCA'].shift(-1)
data = data.dropna()
X = data.drop(columns=['Data Referência', 'Valor IPCA', 'IPCA_Next'])
y = data['IPCA_Next']

2025-07-22 00:34:30,973 - INFO - Preparando dados...


In [7]:
# Divisão treino/teste (80/20, mantendo ordem temporal)
train_size = int(len(data) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]
logging.info(f"Tamanho do conjunto de treino: {len(X_train)} observações")
logging.info(f"Tamanho do conjunto de teste: {len(X_test)} observações")

2025-07-22 00:34:33,360 - INFO - Tamanho do conjunto de treino: 87 observações
2025-07-22 00:34:33,362 - INFO - Tamanho do conjunto de teste: 22 observações


In [8]:
# Feature Selection com Decision Tree
logging.info("Realizando feature selection com Decision Tree...")
dt_selector = DecisionTreeRegressor(
    max_depth=6, min_samples_split=2, min_samples_leaf=3, random_state=42
)
dt_selector.fit(X_train, y_train)
importances = dt_selector.feature_importances_
feature_names = X.columns

2025-07-22 00:34:35,936 - INFO - Realizando feature selection com Decision Tree...


In [9]:
# Selecionar variáveis com importância >= 0.005
importance_threshold = 0.005
selected_features = feature_names[importances >= importance_threshold].tolist()
logging.info(f"Variáveis selecionadas ({len(selected_features)}): {selected_features}")

2025-07-22 00:34:39,271 - INFO - Variáveis selecionadas (6): ['IPCA Mês -11', 'IPCA Mês -10', 'IPCA Mês -1', 'PETROLEO Mês -7', 'IPP NOVO Mês -9', 'DESEMPREGO Mês 0']


In [10]:
# Filtrar conjuntos de treino e teste
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

In [11]:
# Definir modelos e grades de hiperparâmetros
models = {
    'Linear Regression': LinearRegression(),
    'SVR': SVR(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42)
}

param_grids = {
    'Linear Regression': {},  # Sem hiperparâmetros para otimizar
    'SVR': {
        'C': [0.01, 0.1, 1.0, 10.0, 100.0],
        'epsilon': [0.001, 0.01, 0.1, 0.2, 0.5],
        'kernel': ['rbf', 'linear', 'poly'],
        'degree': [2, 3]
    },
    'Decision Tree': {
        'max_depth': [5, 6, 7, 8, 9],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 3]
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'max_depth': [2, 3, 4, 5],
        'min_samples_split': [2, 5, 10]
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [4, 5, 6, 7, 8],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 3]
    }
}

In [12]:
# Configurar validação cruzada temporal
tscv = TimeSeriesSplit(n_splits=5)

In [13]:
# Treinar e avaliar modelos com GridSearchCV
results = []
predictions = {}
best_params = {}

for name, model in models.items():
    logging.info(f"Otimizando {name} com variáveis selecionadas...")
    try:
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grids[name],
            cv=tscv,
            scoring='neg_mean_squared_error',
            n_jobs=1
        )
        grid_search.fit(X_train_selected, y_train)
        
        # Armazenar melhores parâmetros
        best_params[name] = grid_search.best_params_
        logging.info(f"Melhores parâmetros para {name}: {best_params[name]}")
        
        # Treinar modelo com melhores parâmetros
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test_selected)
        
        # Calcular métricas
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Armazenar resultados
        results.append({
            'Model': name,
            'RMSE': rmse,
            'MAE': mae,
            'MAPE (%)': mape,
            'R2': r2
        })
        
        # Armazenar previsões
        predictions[name] = y_pred
    except Exception as e:
        logging.error(f"Erro ao otimizar {name}: {str(e)}")
        continue

2025-07-22 00:34:52,300 - INFO - Otimizando Linear Regression com variáveis selecionadas...
2025-07-22 00:34:52,355 - INFO - Melhores parâmetros para Linear Regression: {}
2025-07-22 00:34:52,363 - INFO - Otimizando SVR com variáveis selecionadas...
2025-07-22 00:35:03,899 - INFO - Melhores parâmetros para SVR: {'C': 10.0, 'degree': 2, 'epsilon': 0.01, 'kernel': 'linear'}
2025-07-22 00:35:03,904 - INFO - Otimizando Decision Tree com variáveis selecionadas...
2025-07-22 00:35:04,732 - INFO - Melhores parâmetros para Decision Tree: {'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 2}
2025-07-22 00:35:04,738 - INFO - Otimizando Gradient Boosting com variáveis selecionadas...
2025-07-22 00:36:03,808 - INFO - Melhores parâmetros para Gradient Boosting: {'learning_rate': 0.05, 'max_depth': 4, 'min_samples_split': 10, 'n_estimators': 300}
2025-07-22 00:36:03,814 - INFO - Otimizando Random Forest com variáveis selecionadas...
C:\Users\JoãoEmileXimenesMuri\AppData\Roaming\Python\Pyth

In [14]:
# Exibir melhores parâmetros
print("\nMelhores Hiperparâmetros:")
for name, params in best_params.items():
    print(f"{name}: {params}")


Melhores Hiperparâmetros:
Linear Regression: {}
SVR: {'C': 10.0, 'degree': 2, 'epsilon': 0.01, 'kernel': 'linear'}
Decision Tree: {'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 2}
Gradient Boosting: {'learning_rate': 0.05, 'max_depth': 4, 'min_samples_split': 10, 'n_estimators': 300}
Random Forest: {'max_depth': 8, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


In [15]:
# Criar DataFrame com resultados
results_df = pd.DataFrame(results)
print("\nResultados dos Modelos (com Feature Selection e Hiperparâmetros Otimizados):")
print(results_df)


Resultados dos Modelos (com Feature Selection e Hiperparâmetros Otimizados):
               Model      RMSE       MAE   MAPE (%)        R2
0  Linear Regression  0.110760  0.086118  23.579420  0.822426
1                SVR  0.111989  0.083958  21.615759  0.818463
2      Decision Tree  0.113943  0.080644  21.459812  0.812073
3  Gradient Boosting  0.085188  0.071864  18.558329  0.894955
4      Random Forest  0.113918  0.098869  26.551961  0.812156


In [16]:
# Identificar o melhor modelo (menor RMSE)
if not results_df.empty:
    best_model = results_df.loc[results_df['RMSE'].idxmin()]
    print("\nMelhor Modelo (baseado no RMSE):")
    print(best_model)
else:
    print("\nNenhum modelo foi otimizado com sucesso.")


Melhor Modelo (baseado no RMSE):
Model       Gradient Boosting
RMSE                 0.085188
MAE                  0.071864
MAPE (%)            18.558329
R2                   0.894955
Name: 3, dtype: object
